# Black boxes

(MyHDL extensions by section5)

Like in Verilog/VHDL, a logic entity that...
  * is 'closed source' (a net list, ...)
  * is resolved at a later stage
  * is a FPGA primitive where only a simulation model might apply
  
is treated and abstracted as a **Black Box**.

## Development aspect

Strategy of **Divide et impera**:
  * Give developer maximum control over what logic should be created
  * ... while providing means to preserve portability
  * Target abstraction: Separate target parameters and implementation rules from design
  * ... but keep code in one place
  * Allow to switch target using one configuration variable
  * ... or by calling an implementation method

## The MyHDL `@blackbox` API

* Allows wrapping external `V*HDL`-modules
* Allows to specify a simplified simulation model for a (closed source) IP module

* [Generator classes](#Generator-classes), allowing elegant 'inline' constructions, as
  * Special operators, like Unary Reduction (no VHDL-built-in!), bit field data types or Gray-Encoder
  * Instance of synthesis-specific constructs

* [Parametrizeable implementations](#Parametrizeable-top-level-design) of top level design projects:
  * (Portable) Hardware projects for multiple FPGA architectures
  * Specific Co-Simulation variants
  * Software-Implementations of hardware IP (simulation executables)

* Future music:
  * Bulk signals (automated signal class to HDL record wrapping for complex signals), like busses, port arrays, ..

## Standard 'external' Blackbox

`@autowrap`: A `@blackbox` unit that automatically wraps the function interface (as in VHDL terms: into a component declaration).

* Does not synthesize any of its behavioural description
* Creates a module stub for the synthesis tool (resolved at a later time)
* Returns a simulator model for simulation

In [1]:
import config

### An on-chip oscillator

..with a output only.

In [2]:
from synthesis.yosys.autowrap import autowrap

@autowrap
def OC_OSC(OSC, **parameter):
    "On-Chip oscillator"

    # half period in pico seconds
    C_OSCP = 1613
    oscb = Signal(bool(0))
    
    @instance
    def sim():
        while 1:
            yield delay(C_OSCP)
            oscb.next = not oscb

    @always_comb
    def assign():
        OSC.next = oscb

    return sim, assign

### More Blackbox examples

* [Generator classes](genclasses.ipynb)

### Next: [Hardware programming](hwprog_setup.ipynb)